In [1]:
import os
import shutil

# Directorio de entrada donde están las imágenes
input_folder = "/home/andres/ambientesVirtuales/PruebasML/Practicas/DatosAProcesar/Train"

# Directorio de salida donde se crearán las subcarpetas
output_folder = "/home/andres/ambientesVirtuales/PruebasML/Practicas/DatosAProcesar/"

# Nombres de las subcarpetas
categories = [
    "dia", "doctor", "dulce", "granja", "mecanico",
    "montaña", "noche", "playa", "saludable", "domestico", "salvaje"
]

# Crear las subcarpetas
for category in categories:
    category_path = os.path.join(output_folder, category)
    os.makedirs(category_path, exist_ok=True)  # Crear la carpeta si no existe

# Mover las imágenes a las subcarpetas según su categoría
for filename in os.listdir(input_folder):
    if filename.endswith((".jpg", ".png", ".jpeg")):  # Filtro para archivos de imagen
        file_path = os.path.join(input_folder, filename)
        
        # Identificar la categoría de la imagen según su nombre
        for category in categories:
            if category in filename.lower():  # Verifica si el nombre de la categoría está en el nombre del archivo
                target_path = os.path.join(output_folder, category, filename)
                shutil.move(file_path, target_path)  # Mover el archivo
                print(f"Moviendo {filename} a {category}")
                break

print("Proceso completado.")


Moviendo domestico746.jpg a domestico
Moviendo granja820.jpg a granja
Moviendo doctor281.jpg a doctor
Moviendo granja700.jpg a granja
Moviendo salvaje741.jpg a salvaje
Moviendo playa760.jpg a playa
Moviendo playa383.jpg a playa
Moviendo salvaje208.jpg a salvaje
Moviendo saludable116.jpg a saludable
Moviendo montaña692.jpg a montaña
Moviendo salvaje688.jpg a salvaje
Moviendo granja637.jpg a granja
Moviendo domestico300.jpg a domestico
Moviendo mecanico334.jpg a mecanico
Moviendo granja305.jpg a granja
Moviendo dulce504.jpg a dulce
Moviendo salvaje520.jpg a salvaje
Moviendo montaña78.jpg a montaña
Moviendo mecanico150.jpg a mecanico
Moviendo granja613.jpg a granja
Moviendo dulce579.jpg a dulce
Moviendo granja507.jpg a granja
Moviendo montaña687.jpg a montaña
Moviendo playa737.jpg a playa
Moviendo granja70.jpg a granja
Moviendo playa79.jpg a playa
Moviendo salvaje1177.jpg a salvaje
Moviendo saludable321.jpg a saludable
Moviendo domestico749.jpg a domestico
Moviendo granja332.jpg a granja


Data aumentation

In [2]:
import albumentations as A
import cv2
import random
import os

# Número de imágenes por clase en tu dataset
class_counts = {
    "dia": 487,
    "doctor": 722,
    "domestico": 921,
    "dulce": 534,
    "granja": 652,
    "mecanico": 492,
    "montaña": 706,
    "noche": 740,
    "playa": 845,
    "saludable": 506,
    "salvaje": 1129
}

# Definir el número objetivo para balancear después de aumentar "salvaje"
target_count = max(class_counts.values())  # Queremos que todas las clases tengan el mismo número máximo de imágenes

def random_resize():
    size = random.randint(256, 640)
    return A.Compose([
        A.LongestMaxSize(max_size=size),  # Redimensiona preservando la relación de aspecto
        A.PadIfNeeded(min_height=size, min_width=size, border_mode=0),  # Completa el tamaño si es necesario
    ])

# Transformaciones con tamaños aleatorios
transform = A.Compose([
    random_resize(),  # Tamaño aleatorio entre 256 y 640
    A.HorizontalFlip(p=0.5),  # Voltear horizontalmente
    A.RandomScale(scale_limit=0.2, p=0.7),  # Cambios de escala
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),  # Desplazamientos y rotaciones
    A.RandomBrightnessContrast(p=0.3),  # Ajuste de brillo y contraste
    A.HueSaturationValue(p=0.3),  # Cambios de color
])

# Función para aumentar imágenes
def augment_and_save(image_path, transform, output_dir, num_augmented):
    # Cargar la imagen
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Nombre base del archivo de imagen
    filename = os.path.splitext(os.path.basename(image_path))[0]

    # Generar y guardar imágenes aumentadas
    for i in range(num_augmented):
        augmented = transform(image=image)
        augmented_image = augmented["image"]

        # Guardar la imagen aumentada con un nombre único
        new_filename = f"{filename}_aug_{i+1}.jpg"
        output_path = os.path.join(output_dir, new_filename)
        augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, augmented_image_bgr)
        print(f"Imagen aumentada guardada en: {output_path}")

# Directorios base
base_input_dir = '/home/andres/ambientesVirtuales/PruebasML/Practicas/DatosAProcesar'
base_output_dir = '/home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/nuevasImagenes'

# Aumentar n veces las imágenes de "salvaje"
class_to_augment = "salvaje"
augmentations_per_image = 1

input_dir = os.path.join(base_input_dir, class_to_augment)
output_dir = os.path.join(base_output_dir, class_to_augment)

# Crear el directorio de salida si no existe
os.makedirs(output_dir, exist_ok=True)

print(f"Aumentando la clase '{class_to_augment}' 1 veces por imagen.")

image_filenames = [f for f in os.listdir(input_dir) if f.endswith((".jpg", ".png", ".jpeg"))]
for image_filename in image_filenames:
    image_path = os.path.join(input_dir, image_filename)
    augment_and_save(image_path, transform, output_dir, augmentations_per_image)

# Actualizar el conteo de "salvaje"
class_counts[class_to_augment] += len(image_filenames) * augmentations_per_image

# Determinar nuevo número objetivo después de aumentar "salvaje"
target_count = max(class_counts.values())

# Balancear las demás clases
for class_name, current_count in class_counts.items():
    input_dir = os.path.join(base_input_dir, class_name)
    output_dir = os.path.join(base_output_dir, class_name)

    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    # Determinar cuántas imágenes necesitamos para balancear
    images_needed = target_count - current_count
    if images_needed <= 0:
        print(f"La clase '{class_name}' ya está balanceada.")
        continue

    # Calcular cuántas aumentaciones por imagen necesitamos
    image_filenames = [f for f in os.listdir(input_dir) if f.endswith((".jpg", ".png", ".jpeg"))]
    num_images = len(image_filenames)
    augmentations_per_image = max(1, images_needed // num_images)

    print(f"Procesando clase '{class_name}': {current_count} imágenes actuales, generando {images_needed} nuevas imágenes.")

    # Aplicar aumentaciones
    for image_filename in image_filenames:
        image_path = os.path.join(input_dir, image_filename)
        augment_and_save(image_path, transform, output_dir, augmentations_per_image)

print("Proceso completado. Todas las clases están balanceadas.")


/home/andres/ambientesVirtuales/PruebasML/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.0 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Aumentando la clase 'salvaje' 1 veces por imagen.
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/nuevasImagenes/salvaje/salvaje741_aug_1.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/nuevasImagenes/salvaje/salvaje208_aug_1.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/nuevasImagenes/salvaje/salvaje688_aug_1.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/nuevasImagenes/salvaje/salvaje520_aug_1.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/nuevasImagenes/salvaje/salvaje1177_aug_1.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/nuevasImagenes/salvaje/salvaje662_aug_1.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation

In [3]:
'''import albumentations as A
import cv2
import os
import random

# Directorios de entrada y su número correspondiente de aumentaciones
input_directories = {
    "dia": 1,
    "doctor": 7,
    "domestico": 1,
    "dulce": 1,
    "granja": 9 ,
    "mecanico": 1,
    "montaña": 23,
    "noche": 30,
    "playa": 27,
    "saludable": 1,
    "salvaje": 7
}

# Definir las transformaciones que deseas aplicar
transform = A.Compose([
    A.RandomRotate90(),  # Rotación aleatoria de 90 grados
    A.HorizontalFlip(p=0.5),  # 50% de probabilidad de hacer flip horizontal
    A.VerticalFlip(p=0.5),  # 50% de probabilidad de hacer flip vertical
    A.Transpose(),

    # Ajustes de color y brillo
    A.OneOf([
        A.RandomBrightnessContrast(),
        A.HueSaturationValue(),
        A.RGBShift(),
    ], p=0.5),

    # Aplicar desenfoques aleatorios
    A.OneOf([
        A.Blur(blur_limit=4),
        A.MotionBlur(blur_limit=4),
        A.MedianBlur(blur_limit=4),
    ], p=0.5),

    # Agregar reducciones y recortes de tamaño
    A.OneOf([
        A.RandomResizedCrop(height=256, width=256, scale=(0.8, 1.0), p=0.5),  # Recorte y redimensionamiento aleatorio
        A.CenterCrop(height=256, width=256, p=0.5),  # Recorte centrado
    ], p=0.5),

    # Asegurarse de redimensionar todas las imágenes a un tamaño estándar
    A.Resize(height=256, width=256),
])


# Función para aumentar y guardar las imágenes
def augment_and_save(image_path, transform, output_dir, num_augmented):
    # Cargar imagen
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Nombre base del archivo de imagen
    filename = os.path.splitext(os.path.basename(image_path))[0]

    # Generar y guardar imágenes aumentadas
    for i in range(num_augmented):
        # Aplicar transformación
        augmented = transform(image=image)
        augmented_image = augmented["image"]

        # Guardar la imagen aumentada con un nombre único
        new_filename = f"{filename}_aug_{i+1}.jpg"
        output_path = os.path.join(output_dir, new_filename)

        # Convertir la imagen de nuevo a BGR para guardar con OpenCV
        augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, augmented_image_bgr)
        print(f"Imagen aumentada guardada en: {output_path}")

# Ruta base para los directorios de entrada y salida
base_input_dir = '/home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation'  # Cambiar a tu ruta de entrada
base_output_dir = '/home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/nuevasImagenes'  # Cambiar a tu ruta de salida

# Procesar cada directorio
for directory, num_augmented in input_directories.items():
    input_dir = os.path.join(base_input_dir, directory)
    output_dir = os.path.join(base_output_dir, directory)

    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    print(f"Procesando directorio: {input_dir} con {num_augmented} aumentaciones por imagen")

    # Aplicar aumentación a cada imagen en el directorio
    for image_filename in os.listdir(input_dir):
        image_path = os.path.join(input_dir, image_filename)
        if os.path.isfile(image_path):
            augment_and_save(image_path, transform, output_dir, num_augmented)
'''

/home/andres/ambientesVirtuales/PruebasML/lib/python3.10/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `dict[str, any]` but got `UniformParams` with value `UniformParams(noise_type=...6, 0.0784313725490196)])` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/home/andres/ambientesVirtuales/PruebasML/lib/python3.10/site-packages/albumentations/augmentations/blur/functional.py:149: UserWarning: blur_limit: Non-zero kernel sizes must be odd. Range (3, 4) automatically adjusted to (3, 5).
  result = _ensure_odd_values(result, info.field_name)


Procesando directorio: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/doctor con 7 aumentaciones por imagen
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/doctor/manguito_aug_1.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/doctor/manguito_aug_2.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/doctor/manguito_aug_3.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/doctor/manguito_aug_4.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/doctor/manguito_aug_5.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/doctor/manguito_aug_6.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/d